# Handling Graphs

In PyG a single graph is described by an instance of `torch_geometric.data.Data`.
By defaults, it can handles the following attributes:

- `data.x`: Node feature matrix with shape `[num_nodes, num_node_features]`
- `data.edge_index`: Graph connectivity in [COO](https://pytorch.org/docs/stable/sparse.html#sparse-coo-docs)(sparse representation) format with shape `[2, num_edges]` and type `torch.long`
- `data.edge_attr`: Edge feature matrix with shape `[num_edges, num_edge_features]`
- `data.y`: Target to train against (may have arbitrary shape), e.g., node-level targets of shape `[num_nodes, *]` or graph-level targets of shape `[1, *]`
- `data.pos`: Node position matrix with shape `[num_nodes, num_dimensions]`

None of these attributes are required and we can also add new attributes.

A basic example of how to create a graph is the following:

In [16]:
import torch
from torch_geometric.data import Data

edge_index: torch.Tensor = torch.tensor([[0, 1, 1, 2],
                                         [1, 0, 2, 1]], dtype=torch.long)
x: torch.Tensor = torch.tensor([[-1], [0], [1]], dtype=torch.float)
data: Data = Data(x=x, edge_index=edge_index)
print(data)
print(f"Node Features: {data.x.T}")
print(f"Edges:\n{data.edge_index}")

Data(x=[3, 1], edge_index=[2, 4])
Node Features: tensor([[-1.,  0.,  1.]])
Edges:
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])


This will create a graph with $3$ nodes with $1$ feature each and $4$ edges.
Note that `edge_index` is created with $2$ separate `list`, the first is for `source` and the second is for `destination`.
If we want to create `edge_index` from a list of index tuple we will do the following using `contiguos()`

In [2]:
edge_index: torch.Tensor = torch.tensor([[0, 1],
                                         [1, 0],
                                         [1, 2],
                                         [2, 1]], dtype=torch.long)
x: torch.Tensor = torch.tensor([[-1], [0], [1]], dtype=torch.float)
data = Data(x=x, edge_index=edge_index.t().contiguous())
print(data)
print(f"Node Features: {data.x.T}")
print(f"Edges:\n{data.edge_index}")

Data(x=[3, 1], edge_index=[2, 4])
Node Features: tensor([[-1.,  0.,  1.]])
Edges:
tensor([[0, 1, 1, 2],
        [1, 0, 2, 1]])


The result is the same.

It is necessary that the elements in `edge_index` only hold indices in the range `{ 0, ..., num_nodes - 1}`.
This can be checked using `validate()` on the `Data` object

In [3]:
print(f"Graph is valid: {data.validate(raise_on_error=True)}")

Graph is valid: True


Since a `Data` object is composed by Tensors, we can tranfer the graph on GPU

In [4]:
device = torch.device("cuda:0")
data.to(device)
print(f"Features on: {data.x.device}")
print(f"Edges on: {data.edge_index.device}")

Features on: cuda:0
Edges on: cuda:0


# Importing a dataset

PyG offers a wide range of ready to use datasets in the `torch_geometric.datasets` package. A full list of datasets can be found [here](https://pytorch-geometric.readthedocs.io/en/latest/modules/datasets.html).
For this example we will use `MNISTSuperpixels`, a graph variant of the common `MNIST` dataset.

In [5]:
from torch_geometric.datasets import MNISTSuperpixels

MNIST_PATH = "../datasets/MNISTSuperpixel"
dataset = MNISTSuperpixels(root=MNIST_PATH)

We can get some basic information about the dataset

In [6]:
print(dataset)
print(f"Classes: {dataset.num_classes}")
print(f"Node features: {dataset.num_node_features}")
print(f"Edge features: {dataset.num_edge_features}")

MNISTSuperpixels(60000)
Classes: 10
Node features: 1
Edge features: 0


We have access to all the dataset and we can get a graph by direct access or slicing a subset.

In [7]:
print(dataset[0])
sliced = dataset[:100]
print(sliced)

Data(x=[75, 1], edge_index=[2, 1399], y=[1], pos=[75, 2])
MNISTSuperpixels(100)


Like in PyTorch, PyG offers a `DataLoader` class in `torch_geometric.loader` module.
A `Dataloader` is used to get batches from a dataset
In our example we can use:

In [15]:
from torch_geometric.loader import DataLoader
from torch_geometric.data import Batch

loader = DataLoader(dataset, batch_size=3, shuffle=True)
batch: Batch = next(iter(loader)) 

print(batch)
print(f"Graphs in the batch: {batch.num_graphs}")
print(batch.batch)

DataBatch(x=[225, 1], edge_index=[2, 4019], y=[3], pos=[225, 2], batch=[225], ptr=[4])
Graphs in the batch: 3
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2])


The `DataLoader` returns an `iterator`. Each element is a subtype `Batch` of `Data` that contains an additional attribute called `batch`, which maps each node to its respective graph in the batch.
In our example we have a `batch_size` of $3$, so each batch contains $3$ graphs. The property `batch` in fact contains integers from $0$ to $batch\_size - 1$ indicating at which graph the node belogns.

This is due the fact that PyG achieves parallelization over a `mini-batch` by creating sparse block diagonal adjacency matrices (defined by `edge_index`) and concatenating feature and target matrices in the node dimension.